## Creating folders for each WARENGRUPPE along with the product data for that WR

This is a one-time used code to structure the data so that products for each product group are in their separate folder along with their prompt format and allowed keys as well as their output csv and json files. This will structure the data so that each product group is treated individually based on its attributes. 

In [ ]:
## Importing libraries and initializing values class to import the relevant variables
import pandas as pd
from values import *
import os

val = Values()

In [ ]:
## loading data from warengruppe list, the products in the webshop and the marketing artikel list that connects the other two datas together
warengr = pd.read_csv(val.wr_filepath,delimiter=';',encoding='latin-1')
shop = pd.read_excel(val.shop_file_path)

## We need the first part of the StoreId number to connect with the marketing artikel list
shop['NUMMER'] = shop['StoreId'].str.split().str[0]
marketing_art = pd.read_csv(val.marketing_artikel,delimiter=';',encoding='latin-1')

### Data Processing

In [ ]:
## changing data types to numeric in marketing artikel list
marketing_art['WARENGR'] = pd.to_numeric(marketing_art['WARENGR'],errors='coerce')
marketing_art.dropna(subset=['WARENGR','NUMMER'],inplace=True)
marketing_art['WARENGR'] = marketing_art['WARENGR'].astype(int)

## cleaning marketing artikel list
marketing_art = marketing_art[marketing_art['WM'].isna()]
marketing_art = marketing_art.dropna(subset='WARENGR')
marketing_art = marketing_art[['WM','NUMMER',"MA_NUMMER",'WARENGR']]
marketing_art = marketing_art.drop_duplicates(subset=['NUMMER'])
marketing_art['NUMMER'] = marketing_art['NUMMER'].str.strip()

## cleaning warengruppe items in warengr list
warengr = warengr[warengr['WAREN_GRP'].str.contains('x',case=False) == False]
warengr['WAREN_GRP'] = pd.to_numeric(warengr['WAREN_GRP'],errors='coerce')
warengr.dropna(subset='WAREN_GRP',inplace=True)
warengr['WAREN_GRP'] = warengr['WAREN_GRP'].astype(int)
warengr = warengr[(warengr['WAREN_GRP'].isna() == False) & (warengr['WAREN_GRP'] != 'nan')]

### Data Structure Creation

In [ ]:
wg_data = []
for num, val in zip(warengr['WAREN_GRP'],warengr['WAREN_GRNA']):
    ## iterating over warengruppe
    artikels_in_wr = marketing_art[marketing_art['WARENGR'] == num]
    ## connecting marketing artikels for each warnegruppe with the shop list
    artikels = pd.merge(artikels_in_wr,shop,how='inner',on="NUMMER")
    artikels = artikels.dropna(subset='WARENGR')
    ## selecting necessary columns
    artikels = artikels[['WM','NUMMER','WARENGR','Name','Beschreibung']]
    ## renaming columns 
    artikels = artikels.rename(columns={'Name':'NAME','Beschreibung':'BESCHREIBUNG'})
    ## data cleaning (removing html codes and unnecessary characters)
    artikels['BESCHREIBUNG'] = artikels['BESCHREIBUNG'].str.replace(' ',' ',regex=True)
    artikels['BESCHREIBUNG'] = artikels['BESCHREIBUNG'].str.replace(r'(<[^>]*>)',' ',regex=True,case=False)
    artikels['BESCHREIBUNG'] = artikels['BESCHREIBUNG'].str.replace(r'&nbsp_|&nbsp;',' ',regex=True)
    artikels['BESCHREIBUNG'] =artikels['BESCHREIBUNG'].str.replace('&Oslash',' ')
    artikels = artikels[artikels['BESCHREIBUNG'].isna() == False]
    ## preparing the product description to have all the information by concatenating both names and description together
    artikels['BESCHREIBUNG'] = artikels['NAME'] + ' - ' + artikels['BESCHREIBUNG']
    artikels = artikels[artikels['WARENGR'].isna() == False]
    artikels = artikels.drop_duplicates()
    wg_data.append([num,val,len(artikels)])

    # if len(artikels) != 0:
    #     path = os.path.join(val.parent_dir, str(num))
    #     print(str(num) + ' ' + str(len(artikels)))
    #     print(path)
        ## Building folders based on the warengruppe numbers
        # os.mkdir(path) 

        # ## inserting the csv file of the products for each warengruppe and adding a prompt placeholder text (to be filled)
        # with open(f'{path}/prompt_{num}.txt','w') as txt:
        #     txt.write("prompt-to-be-filled")
        # artikels.to_csv(f'{path}/{num}.csv',encoding='utf-8', sep=';',index=False)
wg_data

In [ ]:
wg_df = pd.DataFrame(wg_data,columns=['WG_ID','WG_NAME','ITEMS_IN_WG'])
wg_df.to_csv('/Users/maralsheikhzadeh/Documents/Codes/useful-exports/WG_artikels.csv',encoding='utf-8',sep=';',index=False)